In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob

In [ ]:
player_plays = pl.read_csv("data/player_play.csv", null_values=["NA"])
games = pl.read_csv("data/games.csv", null_values=["NA"])
plays = pl.read_csv("data/plays.csv", null_values=["NA"])
players = pl.read_csv("data/players.csv", null_values=["NA"])

In [ ]:
tracking_files = glob.glob("data/tracking_week_*.csv")
tracking_data = pl.concat([pl.read_csv(f, null_values= ["NA"]) for f in tracking_files])

In [ ]:
tracking_data

In [ ]:
# Rename columns in `tracking_data`
tracking_data = tracking_data.rename({"s":"yardsPerSecond", "a":"acceleration","o":"orientation"})

In [ ]:
tracking_data['event'].unique().to_list()

In [ ]:
def calculate_presnap_movement(presnap:pl.DataFrame, snap:pl.DataFrame) -> pl.DataFrame:
    merged = presnap.join(snap, on='displayName', suffix='_snap')
    merged = merged.with_columns(
     (pl.col("x")-pl.col("x_snap")).alias("x_dif"),
     (pl.col("y")-pl.col("y_snap")).alias("y_dif")
    ).sort((pl.col("y_dif")))
    return merged

def determine_sides(game_df:pl.DataFrame, players_df:pl.DataFrame) -> pl.DataFrame:
    offensive_positions = {"QB", "RB", "T", "FB", "G", "C", "WR", "TE"}
    game_with_positions = game_df.join(players_df.select(["displayName", "position"]), on="displayName")
    game_with_sides = game_with_positions.with_columns(
        (pl.col("position").is_in(offensive_positions)).alias("isOffense")
    )
    return game_with_sides

def get_play_outcome(gameId:int, playId:int, plays: pl.DataFrame) -> pl.DataFrame:
    ret = plays.filter(
        (pl.col("gameId")==gameId) &
        (pl.col("playId")==playId)
    )

    return ret

def analyze_man_in_motion(game_df:pl.DataFrame, players_df:pl.DataFrame, plays_df:pl.DataFrame) -> None:
    mim_game_df = game_df.filter(
        (pl.col("event") == "man_in_motion")
    )
    plays = mim_game_df.group_by("playId", maintain_order=True)
    for play_id, play_df in plays:
        pre = play_df.filter(
            (pl.col("frameType") == "BEFORE_SNAP")
        )
        snap = game_df.filter(
            (pl.col("frameType") == "SNAP") &
            (pl.col("playId") == play_id[0])
        )
        play_side_df = determine_sides(pre, players_df)
        play_df = calculate_presnap_movement(play_side_df, snap)
        try:
            off = play_df.filter(
                (pl.col("isOffense") == True)
            ).sort((pl.col("y_dif")), descending=True)
            player= off[0]['displayName'][0]
            dis = round(off[0]['y_dif'][0], 2)
            if dis < 1:
                continue
            dir = off[0]['playDirection'][0]
            pos = off[0]['position'][0]
            print(f"Processed playId: {play_id[0]}\nPresnap movement: {pos} {player} moved {dis} yards to the {dir}.")
            determine_play_outcome(game_df.filter((pl.col("playId")==play_id[0])), plays_df)
        except:
            continue

def analyze_presnap_play(game_df:pl.DataFrame, players_df:pl.DataFrame, plays_df:pl.DataFrame)->None:
    gameId = game_df["gameId"][0]
    game_as_plays = game_df.group_by("playId", maintain_order=True)
    plays_df = plays_df.filter((pl.col("gameId")==gameId))
    play_dict = {}
    for play_id, play_df in game_as_plays:
        b = {"Presnap Movement":False, "Outcome":None,"Quarter":None, "Coverage":None}
        if "man_in_motion" in play_df["event"].unique():
            b["Presnap Movement"] = True
        outcome = get_play_outcome(gameId, play_id[0], plays_df)
        b["Quarter"] = outcome['quarter'][0]
        b["Outcome"] = outcome['playDescription'][0]
        b["Coverage"] = outcome['pff_manZone'][0]
        play_dict[play_id]=b
    
    p = 1
    for k, v in play_dict.items():
        print(f"Play {p}(#{k[0]}):")
        print(f"Presnap:{'movement' if v['Presnap Movement'] else 'no movement.'}")
        print(f"{v['Outcome'][:-1]} facing {v['Coverage'].lower() if v['Coverage'] != None else 'no'} coverage.\n")
        p+=1


def determine_play_outcome(play_df:pl.DataFrame, plays:pl.DataFrame) -> pl.DataFrame:
    # this needs to be verified.
    pre_snap_pos = play_df.filter(
        (pl.col("frameType") == "BEFORE_SNAP") &
        (pl.col("displayName") == "football") &
        (pl.col("event") != "None") 
    )
    post_snap_pos = play_df.filter(
        (pl.col("frameType") == "AFTER_SNAP") &
        (pl.col("displayName") == "football") &
        (pl.col("event") != "None") 
    )
    pre_coord = [pre_snap_pos["x"][0], pre_snap_pos["y"][0]]
    post_coord = [post_snap_pos["x"][0], post_snap_pos["y"][0]]
    if pre_coord == post_coord:
        print("No change in position.")
        return 0
    else:
        print(f"Ball moved from the {round(pre_coord[0])} yard line to the {round(post_coord[0])} yard line for a gain of {round(abs(post_coord[0]-pre_coord[0]), 1)}.")
        r = get_play_outcome(playId=post_snap_pos["playId"][0], gameId=post_snap_pos["gameId"][0], plays=plays)
        print(f"{r['playDescription'][0]}\n")
        return 

def build_field(ax:plt.Axes):
    # Draw field background
    ax.set_facecolor('green')
    ax.set_xlim(0, 120)   # NFL field length including end zones
    ax.set_ylim(0, 53.3)  # NFL field width

    # Add end zones
    ax.add_patch(patches.Rectangle((0, 0), 10, 53.3, linewidth=0, color='gray'))      # Home endzone
    ax.add_patch(patches.Rectangle((110, 0), 10, 53.3, linewidth=0, color='gray'))    # Visitor endzone

    # Add yard lines
    for x in range(10, 111, 10):
        ax.plot([x, x], [0, 53.3], color='white', linewidth=1.5)
        if x < 60:
            ax.text(x, 55, f'{x//10}', ha='center', color='white', fontsize=10)
            ax.text(x, -2, f'{x//10}', ha='center', color='white', fontsize=10)
        elif x > 60:
            ax.text(x, 55, f'{12 - x//10}', ha='center', color='white', fontsize=10)
            ax.text(x, -2, f'{12 - x//10}', ha='center', color='white', fontsize=10)

    # Plot sidelines
    ax.plot([0, 120], [0, 0], color='white', linewidth=2)
    ax.plot([0, 120], [53.3, 53.3], color='white', linewidth=2)
    
    # Set labels and title
    ax.set_xlabel('Field Length (Yards)')
    ax.set_ylabel('Field Width (Yards)')
    ax.set_title(f'Player Positions Pre Snap')


def show_play_presnap_scheme(play_id:int, play_df:pl.DataFrame, players:pl.DataFrame,) -> None:
    init_timestamp = play_df.filter((pl.col("playId")==play_id) &
        (pl.col("frameType")=="BEFORE_SNAP") &
        (pl.col("event") == "line_set"))
    plot_df = determine_sides(play_df, players).filter(
        (pl.col("playId")==play_id) &
        (pl.col("frameType")=="BEFORE_SNAP") &
        (pl.col("event") != "null") &
        (pl.col("time") == init_timestamp["time"][0])
    )
    ball_pos = play_df.filter(
        (pl.col("time")== init_timestamp["time"][0]) & 
        (pl.col("displayName") =="football")
    )[0]
    ball_coors = [ball_pos["x"].to_numpy()[0],ball_pos["y"].to_numpy()[0]]
    off_x_coords = plot_df.filter((pl.col("isOffense")== True))['x'].to_numpy()
    off_y_coords = plot_df.filter((pl.col("isOffense") == True))['y'].to_numpy()
    def_x_coords = plot_df.filter((pl.col("isOffense") == False))['x'].to_numpy()
    def_y_coords = plot_df.filter((pl.col("isOffense") == False))['y'].to_numpy()

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 6.33))

    build_field(ax)

    # Plot x and y coordinates
    ax.scatter(off_x_coords, off_y_coords, color="red", s=50, alpha=0.7, label="Offense")
    ax.scatter(def_x_coords, def_y_coords, color="blue", s=50, alpha=0.7, label="Defense")
    ax.scatter(ball_coors[0], ball_coors[-1], color="black", s=50, alpha=0.7, label="Ball", marker='X')
    ax.legend(loc='upper right')
    plt.show()


def show_play_aftersnap_scheme(play_id:int, play_df:pl.DataFrame, players:pl.DataFrame,) -> None:
    plot_df = determine_sides(play_df, players).filter(
        (pl.col("playId")==play_id) &
        (pl.col("frameType")=="AFTER_SNAP") &
        (pl.col("event") != "null")
    )

    plot_df=plot_df.filter(
        (pl.col("time") == plot_df["time"][-1])
    )
    ball_pos = play_df.filter(
        (pl.col("time")== plot_df["time"][-1]) & 
        (pl.col("displayName") =="football")
    )[0]
    ball_coors = [ball_pos["x"].to_numpy()[0],ball_pos["y"].to_numpy()[0]]
    off_x_coords = plot_df.filter((pl.col("isOffense")== True))['x'].to_numpy()
    off_y_coords = plot_df.filter((pl.col("isOffense") == True))['y'].to_numpy()
    def_x_coords = plot_df.filter((pl.col("isOffense") == False))['x'].to_numpy()
    def_y_coords = plot_df.filter((pl.col("isOffense") == False))['y'].to_numpy()

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 6.33))
    build_field(ax)
    

    # Plot x and y coordinates
    ax.scatter(off_x_coords, off_y_coords, color="red", s=50, alpha=0.7, label="Offense")
    ax.scatter(def_x_coords, def_y_coords, color="blue", s=50, alpha=0.7, label="Defense")
    ax.scatter(ball_coors[0], ball_coors[-1], color="black", s=50, alpha=0.7, label="Ball", marker='X')
    ax.legend(loc='upper right')
    plt.show()



In [ ]:
gameIdArr = tracking_data["gameId"].unique()

In [ ]:
test_game = tracking_data.filter(
    (pl.col("gameId") == 2022102700)
)

In [ ]:
test_game

In [ ]:
analyze_man_in_motion(test_game, players, plays)

In [ ]:
show_play_presnap_scheme(914, test_game, players)

In [ ]:
show_play_aftersnap_scheme(914, test_game, players)

In [ ]:
analyze_presnap_play(test_game, players, plays)